# Imports

In [60]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
from datetime import datetime
import os

# Seed for reproducing

In [61]:
np.random.seed(888)
tf.random.set_seed(404)

# Constants

In [62]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'
NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * CHANNELS

# Get the Data

In [63]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)


# Data Preprocessing

In [64]:
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0
y_train_all = np.eye(NR_CLASSES)[y_train_all]
y_test = np.eye(NR_CLASSES)[y_test]

# Create validation dataset from training data

In [65]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Neural Network Architecture

In [66]:
nr_epochs = 50
learning_rate = 1e-3
n_hidden1 = 512
n_hidden2 = 64

# Define the Model

In [67]:
model = Sequential([
    Flatten(input_shape=(TOTAL_INPUTS,)),
    Dense(n_hidden1, activation='relu'),
    Dropout(0.8),
    Dense(n_hidden2, activation='relu'),
    Dense(NR_CLASSES, activation='softmax')
])

# Compile the Model

In [68]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# TensorBoard Setup

In [69]:
log_dir = os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Training the Model with TensorBoard

In [70]:
history = model.fit(
    x_train, y_train, 
    epochs=nr_epochs, 
    batch_size=1000, 
    validation_data=(x_val, y_val), 
    callbacks=[tensorboard_callback]
)

Epoch 1/50
50/50 [==============================] - 2s 31ms/step - loss: 0.9662 - accuracy: 0.6894 - val_loss: 0.3286 - val_accuracy: 0.9037
Epoch 2/50
50/50 [==============================] - 1s 28ms/step - loss: 0.4064 - accuracy: 0.8769 - val_loss: 0.2334 - val_accuracy: 0.9311
Epoch 3/50
50/50 [==============================] - 1s 27ms/step - loss: 0.3181 - accuracy: 0.9038 - val_loss: 0.1947 - val_accuracy: 0.9412
Epoch 4/50
50/50 [==============================] - 1s 26ms/step - loss: 0.2733 - accuracy: 0.9177 - val_loss: 0.1669 - val_accuracy: 0.9487
Epoch 5/50
50/50 [==============================] - 1s 28ms/step - loss: 0.2489 - accuracy: 0.9262 - val_loss: 0.1510 - val_accuracy: 0.9542
Epoch 6/50
50/50 [==============================] - 1s 24ms/step - loss: 0.2242 - accuracy: 0.9333 - val_loss: 0.1387 - val_accuracy: 0.9580
Epoch 7/50
50/50 [==============================] - 1s 26ms/step - loss: 0.2041 - accuracy: 0.9389 - val_loss: 0.1289 - val_accuracy: 0.9608
Epoch 8/50
50

# Making a Prediction

In [71]:
def make_prediction(image_path):
    img = Image.open(image_path)
    bw = img.convert('L')
    img_array = np.invert(bw)
    test_img = img_array.ravel() / 255.0
    prediction = model.predict(np.array([test_img]))
    return np.argmax(prediction)

# Example of making a prediction

In [72]:
prediction = make_prediction('MNIST/test_img.png')
print(f'Prediction for test image is {prediction}')

1/1 [==============================] - 0s 61ms/step
Prediction for test image is 2


# Testing and Evaluation

In [73]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy:.2%}")

313/313 [==============================] - 1s 3ms/step - loss: 0.0709 - accuracy: 0.9797
Test loss: 0.07092146575450897, Test accuracy: 97.97%
